# Reaction Engineering Lesson: Modeling a Gas-Phase Plug Flow Reactor (PFR)

**Objective:** This lesson provides a detailed walkthrough for designing and simulating a Plug Flow Reactor (PFR) for a gas-phase reaction where the number of moles changes. This is a critical skill, as it requires coupling reaction kinetics with fluid mechanics (volumetric flow rate changes).

**Learning Goals:**
1.  Derive the fundamental design equation for a PFR from a differential mole balance.
2.  Understand why and how to account for changes in volumetric flow rate in gas-phase systems.
3.  Develop a system of Ordinary Differential Equations (ODEs) that describes the molar flow rates of all species along the reactor length.
4.  Solve this system of ODEs using Python's scientific libraries (`SciPy`).
5.  Visualize and interpret the resulting concentration and conversion profiles.

## The PFR Concept

A Plug Flow Reactor (PFR) models the behavior of a reactor where fluid flows as a series of infinitesimally thin 'plugs'. Each plug is a tiny batch reactor that moves down the length of the PFR.

**Key Assumptions:**
*   **No Axial Mixing:** There is no mixing or diffusion in the direction of flow.
*   **Perfect Radial Mixing:** Within any given plug (at a specific axial position), the concentration, temperature, and reaction rate are uniform.
*   **Steady State:** The properties at any given point in the reactor do not change with time.

This means that reactant concentration is high at the inlet and gradually decreases as the fluid flows down the length of the reactor. Our goal is to model this profile.

![PFR Diagram](https://library.fiveable.me/_next/image?url=https%3A%2F%2Fstorage.googleapis.com%2Fstatic.prod.fiveable.me%2Fsearch-images%252F%2522Plug_flow_reactor_configuration_flow_pattern_diagram_continuous_flow_axial_mixing_chemical_engineering%2522-09-chem_reactorsE(2).jpg&w=640&q=75)

## Part 1: The Model - Differential Mole Balance

To model the PFR, we perform a mole balance on a reactant `A` over a differential slice of reactor volume, `dV`.

$$ (\text{Flow of A in at } V) - (\text{Flow of A out at } V+dV) + (\text{Generation of A in } dV) = 0 $$
$$ F_A|_V - F_A|_{V+dV} + r_A \, dV = 0 $$

where $F_A$ is the molar flow rate of A and $r_A$ is the rate of formation of A per unit volume. Rearranging and taking the limit as $dV \rightarrow 0$ gives the fundamental design equation for a PFR in its molar flow form:
$$ \frac{dF_A}{dV} = r_A $$

## Part 2: The Challenge - Gas-Phase Reactions

The rate of reaction, $r_A$, typically depends on **concentration**, $C_A$. For example, for a first-order reaction, $r_A = -k C_A$. The concentration is related to molar flow by:
$$ C_A = \frac{F_A}{v} $$
where $v$ is the **volumetric flow rate**.

**In gas-phase reactions where the number of moles changes, $v$ is NOT constant!** As reactants turn into products, the total number of moles can increase or decrease, changing the volume of the gas (and thus the concentrations of all species).

We can relate the change in volumetric flow to the change in total molar flow ($F_T$) using the ideal gas law (assuming isothermal and isobaric conditions):
$$ v = v_0 \left( \frac{F_T}{F_{T0}} \right) $$
where $v_0$ and $F_{T0}$ are the initial volumetric and total molar flow rates.

**Our Strategy:** We will write our ODEs in terms of molar flows ($F_i$) for every species. Inside our model function, at each step (each `dV`), we will:
1.  Use the current molar flows ($F_i$) to calculate the total molar flow ($F_T$).
2.  Use $F_T$ to calculate the current volumetric flow ($v$).
3.  Use $F_i$ and $v$ to calculate current concentrations ($C_i$).
4.  Use $C_i$ to calculate the reaction rate ($r_A$).
5.  Use $r_A$ to calculate the derivatives ($dF_i/dV$) for the next step of the ODE solver.

## Part 3: Python Implementation

We will model the gas-phase decomposition of Dimethyl Ether (DME) into Methane, Hydrogen, and Carbon Monoxide.
$$ (CH_3)_2O \rightarrow CH_4 + H_2 + CO $$
Let A = DME, B = CH4, C = H2, D = CO. The reaction is $A \rightarrow B + C + D$. The rate law is first order with respect to DME: $r_{DME} = -k C_A$.

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# --- Define Reaction & Feed Parameters ---
k = 0.005         # Reaction rate constant (1/s)
T0 = 800          # Inlet Temperature (K)
P0 = 5.0          # Inlet Pressure (atm)
R = 0.08206       # Gas Constant (L.atm/mol.K)

# Inlet Feed (pure DME)
F_A0 = 10.0       # Molar flow of DME (mol/s)
F_B0 = 0.0
F_C0 = 0.0
F_D0 = 0.0

# Initial total molar flow and concentration
F_T0 = F_A0 + F_B0 + F_C0 + F_D0
C_A0 = F_A0 * P0 / (F_T0 * R * T0) # From C_A0 = y_A0 * P0 / (R*T0)

# Initial volumetric flow rate
v0 = F_T0 * R * T0 / P0

print("Parameters are defined.")

In [ ]:
def pfr_gas_model(V, F):
    """
    Defines the system of ODEs for the gas-phase PFR.
    Args:
        V (float): Current reactor volume (L), the independent variable.
        F (list): State vector of molar flows [F_A, F_B, F_C, F_D].
    """
    # Unpack the molar flows
    F_A, F_B, F_C, F_D = F
    
    # 1. Calculate total molar flow
    F_T = F_A + F_B + F_C + F_D
    
    # 2. Calculate current volumetric flow rate
    v = v0 * (F_T / F_T0) # Assuming T=T0 and P=P0
    
    # 3. Calculate current concentration of reactant A
    C_A = F_A / v
    
    # 4. Calculate the rate of reaction for species A (DME)
    r_A = -k * C_A
    
    # 5. Calculate derivatives for each component based on stoichiometry
    # Reaction: A -> B + C + D
    # r_A = -r_B = -r_C = -r_D
    dFA_dV = r_A
    dFB_dV = -r_A
    dFC_dV = -r_A
    dFD_dV = -r_A
    
    return [dFA_dV, dFB_dV, dFC_dV, dFD_dV]

print("PFR model function is defined.")

In [ ]:
# --- Running the Simulation ---

# Define the integration span (reactor volume from 0 to V_final)
V_final = 5000 # Liters
V_span = (0, V_final)

# Define the initial conditions (feed molar flows)
F_initial = [F_A0, F_B0, F_C0, F_D0]

# Call the ODE solver
solution = solve_ivp(pfr_gas_model, V_span, F_initial, dense_output=True, method='Radau')

# Generate points for a smooth plot
V_plot = np.linspace(V_span[0], V_span[1], 200)
F_plot = solution.sol(V_plot)

print("ODE Simulation complete.")

## Part 4: Analysis and Visualization

Now we plot the results to see what's happening inside our reactor.

In [ ]:
# --- Extract results and calculate conversion ---
F_A_plot = F_plot[0]
F_B_plot = F_plot[1]
F_C_plot = F_plot[2]
F_D_plot = F_plot[3]

# Conversion of reactant A (DME)
X_A_plot = (F_A0 - F_A_plot) / F_A0

# --- Plotting ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot Molar Flows on the primary y-axis (left)
ax1.set_xlabel('Reactor Volume (L)', fontsize=14)
ax1.set_ylabel('Molar Flow Rate (mol/s)', fontsize=14, color='royalblue')
ax1.plot(V_plot, F_A_plot, label='DME (A)', color='royalblue')
ax1.plot(V_plot, F_B_plot, label='Methane (B)', color='firebrick')
ax1.tick_params(axis='y', labelcolor='royalblue')
ax1.legend(loc='upper left')

# Plot Conversion on the secondary y-axis (right)
ax2 = ax1.twinx()
ax2.set_ylabel('Conversion of DME, X', fontsize=14, color='green')
ax2.plot(V_plot, X_A_plot, color='green', linestyle='--', linewidth=3, label='Conversion')
ax2.tick_params(axis='y', labelcolor='green')
ax2.legend(loc='upper right')
ax2.set_ylim(0, 1)

fig.suptitle('PFR Profile for Gas-Phase DME Decomposition', fontsize=16, weight='bold')
fig.tight_layout()
plt.show()

### Interpreting the Plot

This plot is a perfect illustration of the PFR concept:
*   **Molar Flow of DME (Blue):** As expected, the reactant is consumed as it flows down the reactor, so its molar flow decreases.
*   **Molar Flow of Methane (Red):** The product is generated, so its molar flow increases from zero.
*   **Rate of Reaction:** The slope of the DME curve is steepest at the beginning (V=0) and becomes shallower. This is because the reaction rate is highest at the inlet where the reactant concentration is highest.
*   **Conversion (Green):** The conversion steadily increases with reactor volume, showing how more reaction time (volume) leads to a more complete reaction. To achieve ~80% conversion, we would need a reactor of about 3000 L.

## Student Challenges

1.  **Effect of Pressure:** In our model, we assumed the pressure was constant ($P=P_0$). What would happen to the required reactor volume for 80% conversion if you **doubled the inlet pressure `P0`**? Before running, think: how does pressure affect gas concentration and reaction rate?

2.  **Pressure Drop:** A more realistic model would include pressure drop along the length of the reactor, often modeled by the **Ergun Equation**. This is an advanced challenge. Can you add another ODE to the system, $\frac{dP}{dV} = -f(\text{flow properties})$, and use the changing pressure $P(V)$ in your calculation of volumetric flow $v$? 

3.  **Reversible Reaction:** What if the reaction was reversible: $A \rightleftharpoons B + C + D$? You would need to modify the rate law to include a reverse term: $r_A = -k_f C_A + k_r C_B C_C C_D$. How would this change the final conversion profile? Would it still reach 100% if the reactor were infinitely long?